In [8]:
function gx_bound(c_L, c_U, c, c_g, x_now, edge)
    
    #println(f,"Current CELL's LB = ", c_L)
    #println(f,"Current CELL's UB = ", c_U)
    #println(f, "Current interdiction x = ", x_now)
    start_node = edge[:,1]
    end_node = edge[:,2]

    no_node = max(maximum(start_node), maximum(end_node) )
    no_link = length(start_node)


    function getShortestX(state, start_node, end_node, origin, destination)
        _x = zeros(Int, length(start_node))
        _path = enumerate_paths(state, destination)

        for i=1:length(_path)-1
            _start = _path[i]
            _end = _path[i+1]

            for j=1:length(start_node)
                if start_node[j]==_start && end_node[j]==_end
                _x[j] = 1
                break
                end
            end

        end
        _x
    end


    graph = Graph(no_node)
    distmx = Inf*ones(no_node, no_node)

    # Adding links to the graph
    for i=1:no_link
        add_edge!(graph, start_node[i], end_node[i])
        distmx[start_node[i], end_node[i]] = c_g[i]
    end

    # Run Dijkstra's Algorithm from the origin node to all nodes
    state = dijkstra_shortest_paths(graph, origin, distmx)
    label = state.dists
    pred = state.parents
    b_arc = ""
    
    for i = 1: length(state.parents)
        if state.parents[i] != 0 
            b_arc = string(b_arc, "(", state.parents[i], ",", i, ")")
        end
    end
    
    # Retrieving the shortest path
    path = enumerate_paths(state, destination)
    
    #parents = LightGraphs.DijkstraState(state, destination)

    # Retrieving the 'x' variable in a 0-1 vector
    y = getShortestX(state, start_node, end_node, origin, destination)
    #println(f,"y vector:", y)
    
    gx = sum(c_g[i]*y[i] for i = 1:no_link)    
    SP = sum(c[i]*y[i] for i = 1:length(c))
    T = Int64[]
    for i = 1:Len
        if pred[edge[i,2]] == edge[i,1]
            push!(T, 1)
        else
            push!(T, 0)
        end
    end

    y_index = find(y .== 1)

#     println("Minimum Spanning Tree: ", T)
    #println("Shortest path y = ", y)
    #println("Indices of shortest path (edge) = ", y_index)
    #println("Nodes visited =", path)
    #println("Minimum Spanning Tree =", pred)
    #println("Node label =" ,label)

    return y, gx, SP
end

h1 = Model(solver = GurobiSolver(gurobi_env)) # If we want to add # in Gurobi, then we have to turn of Gurobi's own Cuts 

@variable(h1, 1 >= y_h[1:Len]>=0)
#@variable(h, q[1:Len]>=0)

#Setting constraint for start node
leaving = 1 .* (edge[j=1:Len,1] .== origin)
@constraint(h1, sum(y_h[k]*leaving[k] for k=1:Len) == 1)

#Setting constraint for other nodes
for i in all_nodes
    if i != destination
        if i != origin
            incoming = 1 .* (edge[j=1:Len,2] .== i)
            leaving = -1 .* (edge[j=1:Len,1] .== i)
            @constraint(h1, sum(y_h[k]*leaving[k] + y_h[k]*incoming[k] for k=1:Len) == 0)
        end
    end
end

In [12]:
using JuMP 
using Gurobi
using LightGraphs
using DataFrames
using CSV
using TimerOutputs

edge = [1 2; 1 3; 2 3; 2 4; 3 4]
c_L =  [0.0, 8 , 11, 4 , 0]
c_U =  [0.0, 12, 11, 20, 0]
d =  [0.0, 1, 0, 8, 0]
Len = length(c_L)

c = 0.5*(c_L+c_U)
# c = [0, 10, 11, avg, 0]

#STARTING SOLUTION:
yy = zeros(Len)

# yy[1] = 1
# yy[4] = 1


SP_init = sum(yy[i]*c[i] for i = 1:Len)

p = [1.0]
g = [SP_init]
h = [0.0]

origin = 1
destination = 4

last_node = maximum(edge)
all_nodes = collect(1:last_node)

M = zeros(Len)

for i = 1:Len
    M[i] = c_U[i] - c_L[i]
end

case = 0
delta1 = 1
delta2 = 0.01

last_node = maximum(edge)

x_now = zeros(Len)
x_now[2]=1


gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)



#@objective(h, Min, sum((cL_orig[i]+d[i]*x_now[i])*y_h[i] + q[i] for i=1:Len))
#print(h1)
#solve(h1)

# print(x_now)

Academic license - for non-commercial use only


In [13]:
c_g = c + d.*x_now

5-element Array{Float64,1}:
  0.0
 11.0
 11.0
 12.0
  0.0

In [14]:

    
# c = (c_L + c_U)/2
# #println(f,"Current CELL's LB = ", c_L)
# #println(f,"Current CELL's UB = ", c_U)
# #println(f,"Current interdiction x = ", x_now)
# M = c_U - c_L

h2 = copy(h1)

y2 = h2[:y_h]

@variable(h2, q[1:Len]>=0)
for i = 1:Len
    @constraint(h2, q[i] >= c[i] - c_L[:,1][i] - M[i]*(1-y2[i])) #_h[i]))
end

@objective(h2, Min, sum((c_L[i]+d[i]*x_now[i])*y2[i] + q[i] for i=1:Len))

# for i = 1:Len
#     temp = (c_L[i]+d[i]*x_now[i])
#     println(i, " : ", temp)
# end
# print(h2)
solve(h2)

hx = getobjectivevalue(h2)
# println("x_now = ", x_now)
y_g, gx, SP = gx_bound(c_L, c_U, c, c_g, x_now, edge)
# println("g = ", gx, "\ny = ", y_g, "\nSP = ", SP)
# println("y_h = ",getvalue(y2), " \nhx = ", hx)
# println("q = ", getvalue(q))
println("g = ", gx)
println("h = ", hx)

g = 11.0
h = 6.5
